In [1]:
import pandas as pd
import joblib # Importez joblib

df = pd.read_csv(
    "bookParis.csv", 
    sep=';',                
    encoding='utf-8',        # pour gérer les accents
    on_bad_lines='skip'      # ignore les lignes mal formées
)

# Pour Voir les premières lignes de dataset
print("=== Aperçu ===")
print(df.head())

# Vérifier les infos et types
print("\n=== Info ===")
print(df.info())

# Vérifier les valeurs manquantes
print("\n=== Valeurs manquantes ===")
print(df.isnull().sum())

=== Aperçu ===
          Type de document  Prêts 2022  \
0  Bande dessinée jeunesse         814   
1  Bande dessinée jeunesse         602   
2  Bande dessinée jeunesse         989   
3  Bande dessinée jeunesse         688   
4    Bande dessinée adulte         624   

                                              Titre  \
0                    Game over. 6. Sound of silence   
1                                        Note to be   
2         Max et Lili ont peur des images violentes   
3  La Rose écarlate : missions. 3. La dame en rouge   
4                            Les cigares du pharaon   

                      Auteur  Nombre de localisations  Nombre de prêt total  \
0                        NaN                       41                  3559   
1                      Erroc                       36                  2262   
2  Saint-Mars,  Dominique de                       48                  5408   
3         Lyfoung,  Patricia                       46                  2436   
4     

In [2]:
!pip install fastapi uvicorn nest_asyncio joblib


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("bookParis.csv", sep=';')

# Colonnes catégorielles à encoder que on doit conventir
categorical_cols = ['Type de document', 'Auteur']

# Remplacer les NaN par 'Inconnu' pour l'encodage
df['Auteur'] = df['Auteur'].fillna('Inconnu')

# Encoder les colonnes
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    le_dict[col] = le

In [4]:
features = ["Nombre de localisations", "Nombre d'exemplaires", "Type de document_encoded", "Auteur_encoded"]


In [5]:
from sklearn.ensemble import RandomForestRegressor

target = 'Prêts 2022'

# phase d'entainement de modèle 
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(df[features], df[target])

# Faire les prédictions et ajouter la colonne
df['Predicted_Prêts_2022'] = model.predict(df[features])

# Afficher les top 10 livres prédits les plus empruntés
top_predicted = df.sort_values(by='Predicted_Prêts_2022', ascending=False).head(10)
print(top_predicted[['Titre', 'Prêts 2022', 'Predicted_Prêts_2022']])

                                           Titre  Prêts 2022  \
217                       Ça sent la croquette !        1660   
617                               Prout atomique        1607   
420                    A la pêche aux nouilles !        1602   
649                              Karmastrophique        1617   
716                               Funky moumoute        1583   
305                            Big bisous baveux        1582   
462                               Jurassic mamie        1540   
733  Mortelle Adèle. 2. L'enfer c'est les autres        1563   
707                           Tout ça finira mal        1561   
31        Mortelle Adèle.  T.10 .  Choubidoulove        1515   

     Predicted_Prêts_2022  
217               1631.57  
617               1609.25  
420               1594.16  
649               1588.63  
716               1570.64  
305               1561.12  
462               1542.08  
733               1534.02  
707               1520.04  
31             

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Séparer en train et test (80% train/20 test)
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Création et entraîner le modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions sur le test
y_pred = model.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"RMSE du modèle : {rmse:.2f}")

# Ajoute de la prédiction à tout le dataset
df['Predicted_Prêts_2022'] = model.predict(df[features])

# Top 10 livres prédits les plus empruntés
top_predicted = df.sort_values(by='Predicted_Prêts_2022', ascending=False).head(10)
print(top_predicted[['Titre', 'Prêts 2022', 'Predicted_Prêts_2022']])

RMSE du modèle : 84.25
                                           Titre  Prêts 2022  \
617                               Prout atomique        1607   
217                       Ça sent la croquette !        1660   
420                    A la pêche aux nouilles !        1602   
649                              Karmastrophique        1617   
305                            Big bisous baveux        1582   
462                               Jurassic mamie        1540   
716                               Funky moumoute        1583   
31        Mortelle Adèle.  T.10 .  Choubidoulove        1515   
733  Mortelle Adèle. 2. L'enfer c'est les autres        1563   
707                           Tout ça finira mal        1561   

     Predicted_Prêts_2022  
617               1619.58  
217               1619.58  
420               1583.34  
649               1577.03  
305               1568.22  
462               1546.60  
716               1544.62  
31                1537.85  
733               15

In [7]:
# Score R² sur l'ensemble d'entraînement
r2_train = model.score(X_train, y_train)
# Score R² sur l'ensemble de test
r2_test = model.score(X_test, y_test)

print(f"R² Train : {r2_train:.2f}")
print(f"R² Test  : {r2_test:.2f}")

R² Train : 0.97
R² Test  : 0.84


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Meilleurs paramètres trouvés
best_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

best_model.fit(X_train, y_train)

# Prédictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Évaluation
print("R² Train :", r2_score(y_train, y_train_pred))
print("R² Test :", r2_score(y_test, y_test_pred))
print("RMSE Train :", mean_squared_error(y_train, y_train_pred, squared=False))
print("RMSE Test :", mean_squared_error(y_test, y_test_pred, squared=False))



#Visualiser les prédictions vs les vraies valeurs
#Cela te montrera si les erreurs sont réparties de façon homogène ou si ton modèle se trompe sur certains types de livres.

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Vraies valeurs (y_test)")
plt.ylabel("Prédictions")
plt.title("Comparaison des valeurs réelles vs prédictions")
plt.show()


R² Train : 0.9342073552423711
R² Test : 0.8554330949580757


TypeError: got an unexpected keyword argument 'squared'

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor # Le modèle le plus performant
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# CHARGEMENT ET CRÉATION DES DONNÉES PÉRIODIQUES 

try:
    df = pd.read_csv("bookParis.csv", sep=';', encoding='utf-8', on_bad_lines='skip')
except FileNotFoundError:
    print("ATTENTION: Le fichier 'bookParis.csv' n'a pas été trouvé. Utilisation d'un chemin par défaut.")
    df = pd.read_csv("C:/Users/rimha/Downloads/bookParis.csv", sep=';', encoding='utf-8', on_bad_lines='skip')

# Création des colonnes de prêts semestriels pour l'exemple
np.random.seed(42) # Pour la reproductibilité
df['Prêts S1 2022'] = (df['Prêts 2022'] * np.random.uniform(0.4, 0.6, size=len(df))).astype(int)
df['Prêts S2 2022'] = df['Prêts 2022'] - df['Prêts S1 2022']

print(f"La nouvelle cible est 'Prêts S2 2022'. La nouvelle feature est 'Prêts S1 2022'.")


# PRÉ-TRAITEMENT ET ENCODAGE 

# Gestion des valeurs manquantes
df['Auteur'] = df['Auteur'].fillna('Inconnu')

# Encodage des variables catégorielles 
categorical_cols = ['Type de document', 'Auteur']
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    le_dict[col] = le 

# Définir les features qui sont les variables d'entrée et la CIBLE
features = [
    "Prêts S1 2022",              
    "Nombre de localisations", 
    "Nombre d'exemplaires", 
    "Type de document_encoded", 
    "Auteur_encoded"
]
target = 'Prêts S2 2022' # var CIBLE

#  SePARATION TRAIN et de TEST 

X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)


# ENTRAÎNEMENT ET ÉVALUATION DU MODÈLE

model_s2 = GradientBoostingRegressor(
    n_estimators=200, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

model_s2.fit(X_train, y_train)
y_test_pred = model_s2.predict(X_test)

# Calcul des métriques
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test) # Calcul correct de la RMSE

print("\n=== Évaluation pour la Prédiction du Second Semestre (S2) 2022 ===")
print(f"R² Test (S2)  : {r2_test:.3f}")
print(f"RMSE Test (S2) : {rmse_test:.2f}")

# APPLICATION ET ANALYSE 

# Application de la prédiction à l'ensemble du DataFrame
df['Predicted_Prêts_S2_2022'] = model_s2.predict(X)

# Affichage des 10 livres prédits comme les plus empruntés au S2
top_s2_predicted = df.sort_values(by='Predicted_Prêts_S2_2022', ascending=False).head(10)

print("\n=== Top 10 Livres Prédits les Plus Empruntés au Second Semestre 2022 ===")
print(top_s2_predicted[[
    'Titre', 
    'Prêts S1 2022', 
    'Prêts S2 2022', 
    'Predicted_Prêts_S2_2022'
]])

La nouvelle cible est 'Prêts S2 2022'. La nouvelle feature est 'Prêts S1 2022'.

=== Évaluation pour la Prédiction du Second Semestre (S2) 2022 ===
R² Test (S2)  : 0.682
RMSE Test (S2) : 68.76

=== Top 10 Livres Prédits les Plus Empruntés au Second Semestre 2022 ===
                                           Titre  Prêts S1 2022  \
342                             Toi, je te zut !            646   
433                       La rentrée des claques            611   
225      Mortelle Adèle. 3. C'est pas ma faute !            679   
312                    La fille de Vercingétorix            536   
353                          Le papyrus de César            581   
649                              Karmastrophique            807   
733  Mortelle Adèle. 2. L'enfer c'est les autres            788   
462                               Jurassic mamie            845   
404                  Astérix et la Transitalique            621   
617                               Prout atomique            731

In [10]:
# Partie de API FASTAPI PYTHON
import joblib
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.preprocessing import LabelEncoder       

joblib.dump(model, 'ml_model.pkl')
joblib.dump(le_dict, 'label_encoders.pkl')

print("Modèles regénérés avec la version actuelle de scikit-learn.")

Modèles regénérés avec la version actuelle de scikit-learn.


In [11]:
import os
print(os.getcwd())

C:\Users\rimha


In [12]:
import os
print("Le dossier de travail du Notebook est :")
print(os.getcwd())

Le dossier de travail du Notebook est :
C:\Users\rimha


In [15]:


import pandas as pd
import joblib

# 1. Définir les features originales
features_original = [
    "Nombre de localisations", 
    "Nombre d'exemplaires", 
    "Type de document_encoded", 
    "Auteur_encoded"
]

# 2. Recharger les éléments nécessaires pour garantir l'état correct
try:
   
    df = pd.read_csv("bookParis.csv", sep=';', encoding='utf-8', on_bad_lines='skip')
    model = joblib.load('ml_model.pkl')
    le_dict = joblib.load('label_encoders.pkl')
except FileNotFoundError:
    print("Erreur: Le fichier CSV ou les fichiers .pkl sont introuvables. Vérifiez les chemins.")
    # Sortir de la cellule ou lever une exception si les fichiers sont critiques
    raise

# 3. Reproduire le pré-traitement initial sur le DataFrame frais
df['Auteur'] = df['Auteur'].fillna('Inconnu')
df['Type de document_encoded'] = le_dict['Type de document'].transform(df['Type de document'])
df['Auteur_encoded'] = le_dict['Auteur'].transform(df['Auteur'])

# 4. Faire la prédiction en utilisant UNIQUEMENT les features originales
df['Predicted_Prêts_2022'] = model.predict(df[features_original])

# 5. Sélectionner et sauvegarder les top 10
top_predicted = df.sort_values(by='Predicted_Prêts_2022', ascending=False).head(10)

# 6. Préparation et sauvegarde pour l'API
top_books_data = top_predicted[['Titre', 'Auteur', 'Prêts 2022', 'Predicted_Prêts_2022']].to_dict('records')
joblib.dump(top_books_data, 'top_books.pkl')

print("Sauvegarde terminée. Le fichier top_books.pkl est créé et prêt pour l'API.")

Sauvegarde terminée. Le fichier top_books.pkl est créé et prêt pour l'API.


In [17]:
import joblib
import os


FILE_PATH = "C:/Users/rimha/OneDrive/Desktop/top_books.pkl" 

print(f"Tentative de chargement du fichier : {FILE_PATH}")

try:
    # 1. Charger le contenu
    top_books_data = joblib.load(FILE_PATH)
    
    print("\n--- ✅ VÉRIFICATION RÉUSSIE ---")
    
    # 2. Afficher la structure (doit être une liste)
    print(f"Type de donnée chargée : {type(top_books_data)}")
    print(f"Nombre de livres trouvés : {len(top_books_data)}")
    
    # 3. Afficher les 3 premières entrées pour vérifier le contenu
    print("\n--- Contenu (3 premières entrées) : ---")
    for i, book in enumerate(top_books_data[:3]):
        # Afficher uniquement les champs pertinents pour vérification
        print(f"Livre #{i+1}:")
        print(f"  Titre: {book.get('Titre', 'N/A')}")
        print(f"  Auteur: {book.get('Auteur', 'N/A')}")
        print(f"  Prêts Prédits: {book.get('Predicted_Prêts_2022', 'N/A')}")
        print("-" * 20)
        
except FileNotFoundError:
    print(f"\n--- ❌ ERREUR : Le fichier n'a pas été trouvé à l'emplacement : {FILE_PATH} ---")
    print("Vérifiez le chemin.")
    
except Exception as e:
    print(f"\n---  ERREUR LORS DU CHARGEMENT ---")
    print(f"Détail : {e}")

Tentative de chargement du fichier : C:/Users/rimha/OneDrive/Desktop/top_books.pkl

--- ✅ VÉRIFICATION RÉUSSIE ---
Type de donnée chargée : <class 'list'>
Nombre de livres trouvés : 10

--- Contenu (3 premières entrées) : ---
Livre #1:
  Titre: Prout atomique
  Auteur: Mr Tan
  Prêts Prédits: 1619.58
--------------------
Livre #2:
  Titre: Ça sent la croquette !
  Auteur: Mr Tan
  Prêts Prédits: 1619.58
--------------------
Livre #3:
  Titre: A la pêche aux nouilles !
  Auteur: Mr Tan
  Prêts Prédits: 1583.34
--------------------


In [16]:
import os
print("Le dossier de travail du Notebook est :")
print(os.getcwd())

Le dossier de travail du Notebook est :
C:\Users\rimha
